# Быстро строим деревья


Деревья глубины 1 обычно называют решающими пнями. 
Как Вы думаете, какая сложность построения решающего пня?


Представим, что мы решаем задачу регрессии, считаем для одного сплита. 

$$Q = F(R_m) - \frac{N_l}{N_m} F(R_l) - \frac{N_r}{N_m} F(R_r) $$

$$F(R) = \frac{1}{N} \sum_{i=1}^{N}(mean(y) - y_i) ^ 2 $$

$$Q = F(R_m) - \frac{1}{N_m} [ \sum_{i \in R_l}(mean(y_l) - y_i) ^ 2  +  \sum_{i \in R_r}(mean(y_r) - y_i) ^ 2 ]  $$

$F(R_m) и N_m$ от сплита не зависит, можно отбросить. Осталось научиться быстро считать такие выражения:
$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2$ для всех порогов. 

Если делать в лоб, то получится $O(||R||^2)$, нам не нравится, очень долго. Каждый раз пересчитывать довольно глупо, потому что мы точно знаем, какой именно объект при изменении сплита перешел из левой части в правую!




Воспользуемся формулой для дисперсии:
$$D(R) = mean(y ^ 2) -  mean(y) ^ 2 =   \frac{1}{N} \sum_{i \in R} y_i^2 -  \frac{1}{N ^ 2} (\sum_{i \in R} y_i) ^2$$

Но у нас не дисперсия, а дисперсия без деления на число объектов, то есть:
$$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2 =   \sum_{i \in R_k} y_i^2 -  \frac{1}{N_l} (\sum_{i \in R_l} y_i) ^2 $$
Аналогично для правого сплита. 

Вот мы и получили более простой алгоритм подсчета сплита:

1) Отсортировали фичу и таргет по фиче

2) Сначала считаем, что порог это минимальное значение, то есть левый кусок пустой, правый все значения. Храним для каждой из частей
а) Сумму
б) Сумму квадратов
в) Число объектов

3) Взяли следующее значение в отсортированном списке, это значение ушло от правого куска в левое. Обновили а) б) в) в обоих кусках

4) Посчитали  для каждого куска б) - а) ** 2 / в), сложили для обоих кусков. Если эта сумма маеньше лучшего значения, то это лучший сплит из текущих


Реализуйте алгоритм, сравните качество со sklearn.

Если получилось одинаково, Вы молодец!!!

In [97]:
import numpy as np

In [234]:
def get_threshold(feature, y, train_mode=None):
    # YOUR CODE HERE
    xy = []
    N = len(y)
    for j in range(N):
        xy.append([feature[j], y[j]])
    xy = sorted(xy, key=lambda pair: pair[0])
    y = [p[1] for p in xy]
    x = [p[0] for p in xy]
    
    sum_left = 0
    sum_right = sum(y)
    sum_sq_left = 0
    sum_sq_right = sum([j ** 2 for j in y])
    err = sum_sq_right - sum_right ** 2 / N
    idx = N - 1

    for j in range(N):
                
        sum_left += y[j]
        sum_right -= y[j]
        
        sum_sq_left += y[j] ** 2
        sum_sq_right -= y[j] ** 2
        
        cur_err = sum_sq_left - sum_left ** 2 / (j + 1)
        if j < N - 1:
            cur_err += sum_sq_right - sum_right ** 2 / (N - j - 1)
        if cur_err < err:
            err = cur_err
            idx = j

    return err, x[idx]


def train_stump(X, y):
    best_f = None
    best_thr = None
    best_error = float('inf')
    left_mean, right_mean = None, None
    for j in range(X.shape[1]):
        f_score, f_thr = get_threshold(X[:, j], y)
        if f_score < best_error:
            best_error = f_score
            best_f = j
            best_thr = f_thr
    
    left_mean = y[np.where(X[:, best_f] < best_thr)].mean()
    right_mean = y[np.where(X[:, best_f] >= best_thr)].mean()
    
    return best_f, best_thr, left_mean, right_mean


def predict_stump(X, best_f, best_thr, left_mean, right_mean):    
    feat = X[:, best_f]
    y_pred = np.where(feat <= best_thr, left_mean, right_mean)
    return y_pred

    

In [235]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)

In [236]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
regressor = DecisionTreeRegressor(random_state=0).fit(X, y)
print(mean_squared_error(y, regressor.predict(X)))
regressor_stump = DecisionTreeRegressor(random_state=0, max_depth=1).fit(X, y)
print(mean_squared_error(y, regressor_stump.predict(X)))

0.0
46.19909167710848


In [237]:
best_f, best_thr, left_mean, right_mean = train_stump(X, y)
y_pred = predict_stump(X, best_f, best_thr, left_mean, right_mean)
print(mean_squared_error(y, y_pred))

46.2021637866266
